✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

#### Таблица city — это справочник городов. Структура справочника представлена ниже.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ  
city_id	integer	уникальный идентификатор города, первичный ключ  
city_name	text	название города  
state	text	штат, к которому относится город  
population	integer	население города  
area	numeric	площадь города  

#### Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ  
cust_id	integer	уникальный идентификатор клиента, первичный ключ  
cust_name	text	название клиента  
annual_revenue	numeric	ежегодная выручка  
cust_type	text	тип пользователя  
address	text	адрес  
zip	integer	почтовый индекс  
phone	text	телефон  
city_id	integer	идентификатор города, внешний ключ к таблице city  

#### Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ  
driver_id	integer	уникальный идентификатор водителя, первичный ключ  
first_name	text	имя водителя  
last_name	text	фамилия водителя  
address	text	адрес водителя  
zip_code	integer	почтовый индекс водителя  
phone	text	телефон водителя  
city_id	integer	идентификатор города водителя, внешний ключ к таблице city  

#### В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ  
truck_id	integer	Уникальный идентификатор грузовика, первичный ключ  
make	text	Производитель грузовика  
model_year	integer	Дата выпуска грузовика  

#### Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ  
ship_id	integer	уникальный идентификатор доставки, первичный ключ   
cust_id	integer	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer  
weight	numeric	вес посылки  
truck_id	integer	идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck  
driver_id	integer	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver  
city_id	integer	идентификатор города в который совершена доставка, внешний ключ к таблице city  
ship_date	date	дата доставки  

Задание 1.1
→ Укажите название города с максимальным весом единичной доставки.   (Green Bay)
    
    SELECT
        c.city_name,
        MAX(s.weight)
    FROM 
        sql.city c
    JOIN sql.shipment s ON c.city_id = s.city_id 
    GROUP BY c.city_name 
    ORDER BY 2 DESC


→ Сколько различных производителей грузовиков перечислено в таблице truck? (3)

    SELECT
        COUNT(DISTINCT make)
    FROM sql.truck 

→ Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту? (Holger)

    SELECT
        d.first_name,
        c.cust_name,
        COUNT(s.cust_id)
    FROM 
        sql.driver d
    JOIN sql.shipment s ON d.driver_id = s.driver_id
    JOIN  sql.customer c on c.cust_id = s.cust_id
    GROUP BY d.first_name, c.cust_name
    ORDER BY 3 DESC


→ Укажите даты первой и последней по времени доставок в таблице shipment. (08.01.2016   27.12.2017)

    SELECT
        s.ship_date
    FROM 
        sql.shipment s
    GROUP BY s.ship_date 
    ORDER BY 1 DESC

→ Укажите имя клиента, получившего наибольшее количество доставок за 2017 год. (Autoware Inc)

    SELECT
        c.cust_name,
        COUNT(ship_id) ship_number
    FROM 
        sql.customer c
    JOIN sql.shipment s ON c.cust_id = s.cust_id
    WHERE extract(YEAR from s.ship_date) = 2017
    GROUP BY c.cust_name
    ORDER BY 2 DESC



## UNION

### ПРИНЦИП И УСЛОВИЯ РАБОТЫ UNION

✍ Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?


Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

    SELECT 
         book_name object_name, 'книга' object_description 
    FROM 
         public.books

    UNION ALL

    SELECT 
         movie_title, 'фильм' 
    FROM 
         sql.kinopoisk

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:

одинаковый тип данных;  
одинаковое количество столбцов;  
одинаковый порядок столбцов согласно типу данных.  

### ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

UNION выводит только уникальные записи;  
UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.  

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

### СИНТАКСИС

Запрос строится таким образом:

    SELECT
         n columns
    FROM 
         table_1

    UNION ALL

    SELECT 
         n columns
    FROM 
         table_2
    ...

    UNION ALL

    SELECT 
         n columns
    FROM 
         table_n
         
Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

    SELECT
         c.city_id object_name, 'id города' object_type
    FROM 
         sql.city c

    UNION ALL

    SELECT
         d.driver_id other_name, 'id водителя' other_type
    FROM 
         sql.driver d

    UNION ALL

    SELECT
         s.ship_id, 'id доставки'
    FROM 
         sql.shipment s

    UNION ALL

    SELECT
         c.cust_id, 'id клиента'
    FROM 
         sql.customer c

    UNION ALL

    SELECT
         t.truck_id, 'id грузовика'
    FROM 
         sql.truck t
    ORDER BY 1
    
Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

    SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
    UNION ALL
    SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
    ORDER BY 1
    LIMIT 1
    
Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

    SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
    ORDER BY 1
    LIMIT 1
    UNION ALL
    SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
    ORDER BY 1
    LIMIT 1

Вместо результата получим сообщение о синтаксической ошибке: "...syntax error at or near "UNION"..." Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

    (SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
    ORDER BY 1
    LIMIT 1)
    UNION ALL
    (SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
    ORDER BY 1
    LIMIT 1)
Отлично! Мы получили именно то, что хотели.

⚡ Важно! Если платформа не принимает ваш вариант кода, обращайте внимание на выпадающий пункт меню See full output: там иногда могут быть подсказки о причине ошибки.

Задание 2.1
Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck).
Отсортируйте список по названию объекта, а затем — по типу.

     SELECT
          c.city_name object_name, 'city' object_type
     FROM 
          sql.city c

     UNION 

     SELECT
          c.state, 'state'
     FROM 
          sql.city c

     UNION 

     SELECT
          d.first_name, 'driver'
     FROM 
          sql.driver d

     UNION 

     SELECT
          t.make, 'truck'
     FROM 
          sql.truck t
     ORDER BY 1, 2





Задание 2.2

Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы city.  
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

    SELECT c.city_name object_name
    FROM sql.city c
  
    UNION ALL

    SELECT c.state object_name
    FROM sql.city c

    ORDER BY 1


Задание 2.3
Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

    SELECT DISTINCT(c.city_name) object_name
    FROM sql.city c
  
    UNION 

    SELECT DISTINCT(c.state) object_name
    FROM sql.city c

    ORDER BY 1

## UNION и ограничение типов данных

### ПОЧЕМУ ТАК ВАЖЕН ТИП ДАННЫХ?

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.


Давайте напишем запрос, который позволит получить нужный нам результат.

    SELECT 
         c.city_id
    FROM
         sql.city c

    UNION ALL

    SELECT 
         cc.city_name
    FROM
         sql.city cc
         
Вместо результата вы получите сообщение об ошибке: "ERROR: UNION types integer and text cannot be matched". Дело в том, что мы попытались объединить числовой и строковый типы в одной колонке, а это невозможно.

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели column_name::column_type.

Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.


Немного подправим запрос, чтобы получить желаемый результат.

    SELECT 
         c.city_id::text
    FROM
         sql.city c

    UNION ALL

    SELECT 
         cc.city_name
    FROM
         sql.city cc

Задание 3.1

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник.   
Также добавьте столбец с именем водителя и столбец с типом контакта (phone или zip в зависимости от типа).  
Столбцы к выводу: contact, first_name, contact_type.
Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.  

    
     SELECT
          d.phone contact,
          d.first_name first_name, 
          'phone' contact_type
     FROM 
          sql.driver d

     UNION ALL

     SELECT
          d.zip_code::text contact, 
          d.first_name first_name, 
          'zip' contact_type     
     FROM 
          sql.driver d
     ORDER BY 1, 2


## UNION ALL и промежуточные итоги

### ВОЗМОЖНОСТИ UNION

✍ Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

    SELECT
         c.city_name,
         c.population
    FROM
         sql.city c

    UNION ALL

    SELECT
         'total',
         SUM(c.population)
    FROM
         sql.city c
    ORDER BY 2 DESC

Выполнив запрос в Metabase, вы заметите, что сначала выведен итоговый результат с суммой населения всех городов в таблице, а затем по каждому городу в отдельности.

Blank Advanced Problem

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipping.
Не забывайте о единой типизации.
Упорядочьте по убыванию столбца date_period.

     SELECT
         s.ship_date::text date_period,
         COUNT(s.ship_id) cnt_shipping
     FROM
         sql.shipment s
     GROUP BY s.ship_date
     
     UNION ALL

     SELECT
         'total_shipments',
         COUNT(s.ship_id)
     FROM
          sql.shipment s
     ORDER BY 1 DESC

### 5. UNION и дополнительные условия

✍ UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».
Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.

    SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info
    FROM
         sql.driver d
    WHERE d.phone IS NOT NULL

    UNION

    SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info
    FROM
         sql.driver d
    WHERE d.phone IS NULL

Задание 5.1

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:
если в город была осуществлена доставка, то выводим 'доставка осуществлялась';  
если нет — выводим 'доставка не осуществлялась'.  
Столбцы к выводу: city_name, state, shipping_status.  

Отсортируйте в алфавитном порядке по городу, а затем — по штату.

    SELECT
        c.city_name,
        c.state,
        'доставка осуществлялась' shipping_status
    FROM
        sql.city c
    LEFT JOIN sql.shipment s ON c.city_id = s.city_id
    WHERE s.ship_id IS NOT NULL

    UNION

    SELECT
        c.city_name,
        c.state,
        'доставка не осуществлялась' shipping_status
    FROM
        sql.city c
    LEFT JOIN sql.shipment s ON c.city_id = s.city_id
    WHERE s.ship_id IS NULL

    ORDER BY 1, 2




Задание 5.2

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки.

Пусть первый столбец содержит название города, а второй формируется так:

если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

    SELECT
        c.city_name,
        COUNT(s.ship_id) shippings_fake
    FROM
        sql.city c
    JOIN sql.shipment s ON c.city_id = s.city_id
    GROUP BY c.city_name
    HAVING COUNT(s.ship_id) > 10

    UNION

    SELECT
        c.city_name,
        COUNT(s.ship_id)+5 shippings_fake
    FROM
        sql.city c
    JOIN sql.shipment s ON c.city_id = s.city_id
    GROUP BY c.city_name
    HAVING COUNT(s.ship_id) <= 10

    ORDER BY 2 DESC, 1





### 6. UNION и ручная генерация

✍ UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.

    SELECT 
         'a' letter,'1' ordinal_position

    UNION 

    SELECT 
         'b','2'

    UNION 

    SELECT
         'c','3'
         
Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

Задание 6.1

Напишите запрос, который выберет наибольшее из значений:

1000000;  
541;  
-500;  
100.  

     SELECT 
         1000000 number

     UNION 

     SELECT 
         541 number

     UNION 

     SELECT
         -500 number

     UNION 

     SELECT
         100 number
     
    ORDER BY 1
    LIMIT 1

    
    


Задание 6.2

Мы помним, что сортировка для числовых и строковых типов данных отличается.

Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

1000000;  
541;  
-500;  
100.


     SELECT 
         '1000000' number

     UNION 

     SELECT 
         '541' number

     UNION 

     SELECT
         '-500' number

     UNION 

     SELECT
         '100' number
     
    ORDER BY 1 DESC
    LIMIT 1

Задание 6.3

Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

+ ;   
- ; 
= ;  
/ .  


        SELECT   
            '+' number

        UNION  

        SELECT  
            '-' number

        UNION  

        SELECT 
            '=' number

        UNION  

        SELECT 
            '/' number
     
        ORDER BY 1 DESC
        LIMIT 1





### 7. EXCEPT

### ИСКЛЮЧАЕМ ПОВТОРЯЮЩИЕСЯ ДАННЫЕ

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.

    SELECT
         c.city_name
    FROM
         sql.shipment s
    JOIN sql.city c ON s.city_id = c.city_id

    EXCEPT

    SELECT
         cc.city_name
    FROM
         sql.driver d 
    JOIN sql.city cc ON d.city_id=cc.city_id
    ORDER BY 1
    
Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор EXCEPT

Присоединяя с помощью EXEPT мы оставляем записи первого запроса, за исключением тех, которые встречаются во втором запросе.

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

одинаковый тип данных;  
одинаковое количество столбцов;  
одинаковый порядок столбцов согласно типу данных.  
Синтаксис выглядит следующим образом:  

    SELECT 
         n columns
    FROM 
         table_1

    EXCEPT

    SELECT 
         n columns
    FROM 
         table_2
         
Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

Задание 7.1

Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip.
В поле ниже введите запрос, с помощью которого вы решили эту задачу.

    SELECT
         d.zip_code zip
    FROM
         sql.driver d

    EXCEPT

    SELECT
         c.zip zip
    FROM
         sql.customer c
    ORDER BY 1



### 8. INTERSECT

#### ВЫБИРАЕМ ОБЩИЕ ДАННЫЕ

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?
Предположим, нам надо вывести совпадающие по названию города и штаты.

    SELECT 
         c.city_name object_name
    FROM 
         sql.city c

    INTERSECT

    SELECT 
         cc.state
    FROM 
         sql.city cc
    ORDER BY 1
    
Как видим, с помощью оператора INTERSECT мы вывели названия городов и штатов, которые совпадают: New York, Washington и Wyoming. Присмотримся к нему внимательнее.

Чтобы лучше понять, как работает этот оператор, вновь обратимся к диаграмме Венна: INTERSECT оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.

Присоединяя с помощью  INNERSECT мы оставляем те записи, которые встречаются как в первом,так и во втором запросе.

Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

    SELECT 
         n columns
    FROM 
         table_1

    INTERSECT

    SELECT 
         n columns
    FROM 
         table_2

Задание 8.1

Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

    SELECT 
        c.city_id 
    FROM 
        sql.customer c

    INTERSECT

    SELECT 
        s.city_id 
    FROM 
        sql.shipment s

    INTERSECT

    SELECT 
        d.city_id 
    FROM 
        sql.driver d



Задание 8.2

Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

    SELECT 
        c.zip zip
    FROM 
        sql.customer c

    INTERSECT

    SELECT 
        d.zip_code
    FROM 
        sql.driver d

### 9. Итоги. Закрепление знаний

✍ В этом модуле вы освоили ещё несколько возможностей из функционала SQL.

Теперь вы умеете:

присоединять таблицы «снизу», одну под другой;  
исключать или, наоборот, выбирать повторяющиеся записи из двух таблиц;  
вручную добавлять записи в таблицу.  
Запишем структуру запроса с учётом полученных знаний.  

    SELECT 
         N columns
    FROM 
         table_1

    UNION / UNION ALL / EXCEPT / INTERSECT 

    SELECT 
         N columns
    FROM 
         table_2

ДОПОЛНИТЕЛЬНЫЕ СВЕДЕНИЯ

Напоследок разберём несколько вопросов, которые часто задают на собеседованиях ↓



Задание 9.1

Выведите города с максимальным и минимальным весом единичной доставки.
Столбцы к выводу — city_name, weight.

    (SELECT
        c.city_name,
        MAX(s.weight) weight
    FROM 
        sql.city c
    JOIN sql.shipment s ON c.city_id = s.city_id 
    GROUP BY c.city_name 
    ORDER BY 2 DESC
    LIMIT 1)

    UNION

    (SELECT
        c.city_name,
        MIN(s.weight) weight
    FROM 
        sql.city c
    JOIN sql.shipment s ON c.city_id = s.city_id 
    GROUP BY c.city_name 
    ORDER BY 2 ASC
    LIMIT 1)


Задание 9.2

Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id).
Столбец к выводу — mutual_id.
Отсортируйте по возрастанию.

    SELECT 
         c.cust_id mutual_id
    FROM 
         sql.customer c

    INTERSECT

    SELECT 
         s.ship_id mutual_id
    FROM 
         sql.shipment s
    ORDER BY 1 ASC


Задание 9.3

Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'.
Столбцы к выводу — object_name, object_description.
Отсортируйте по названию в алфавитном порядке.


    SELECT
        DISTINCT(c.cust_name) object_name,
        'КЛИЕНТ' object_description
    FROM
        sql.customer c
    WHERE c.cust_type='manufacturer'

    UNION

    SELECT
        DISTINCT(t.make) object_name,
        'ГРУЗОВИК' object_description
    FROM
        sql.truck t

    ORDER BY 1
